In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from train_csv import train_dual_encoder_csv

train_df = pd.read_csv("/content/drive/MyDrive/CapstoneProject/Datasets/Flickr8k_text/flickr8k_train.csv")

train_dual_encoder_csv(
    train_csv="/content/drive/MyDrive/CapstoneProject/Datasets/Flickr8k_text/flickr8k_train.csv",
    image_dir="/content/drive/MyDrive/CapstoneProject/Datasets/Flickr8k_images/",
    vocab_path="/content/vocab.pkl",
    output_path="/content/dual_cont_encoder_csv.pth"
)

In [ ]:
from ret_train_hf import train_dual_encoder_hf
from datasets import load_dataset

ds = load_dataset("jxie/flickr8k")

train_dual_encoder_hf(
    dataset=ds["train"],
    vocab_path="/content/vocab-HF-8K.pkl",
    output_path="/content/dual_hf_encoder.pth",
    caption_field="caption_0"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/687 [00:00<?, ?B/s]

data/train-00000-of-00002-2f8f6bfa852eac(…):   0%|          | 0.00/414M [00:00<?, ?B/s]

data/train-00001-of-00002-2173151d8cd6c7(…):   0%|          | 0.00/414M [00:00<?, ?B/s]

data/validation-00000-of-00001-7025a2b59(…):   0%|          | 0.00/138M [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 157MB/s]


Epoch 1/5, Loss: 3.0115
Epoch 2/5, Loss: 1.8471
Epoch 3/5, Loss: 1.2128
Epoch 4/5, Loss: 0.8399
Epoch 5/5, Loss: 0.6169
Model saved to /content/dual_hf_encoder.pth


In [1]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch

from dataloader_hf import FlickrDataset, collate_fn
from cap_model_m1 import ResNetLSTMCaptionModel
from datasets import load_dataset

ds = load_dataset("jxie/flickr8k")

dataset = FlickrDataset(
    ds["train"],
    vocab_path="/content/vocab-HF-8K.pkl"
)

loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)

images, captions, lengths = next(iter(loader))

model = ResNetLSTMCaptionModel(
    vocab_size=len(dataset.word2int)
)

outputs = model(images, captions, lengths)

print("Images:", images.shape)
print("Captions:", captions.shape)
print("Outputs:", outputs.shape)

criterion = nn.CrossEntropyLoss(ignore_index=dataset.pad_idx)
loss = criterion(
    outputs.reshape(-1, outputs.size(-1)),
    captions[:, 1:].reshape(-1)
)

print("Sanity loss:", loss.item())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 176MB/s]


Images: torch.Size([2, 3, 224, 224])
Captions: torch.Size([2, 13])
Outputs: torch.Size([2, 12, 3869])
Sanity loss: 8.245420455932617


In [ ]:
from datasets import load_dataset
from dataloader_hf import FlickrDataset
import torch

from cap_train_hf import fit_model_hf
from cap_model_m1 import ResNetLSTMCaptionModel   # your M1 model

ds = load_dataset("jxie/flickr8k")

train_ds = ds["train"]
val_ds   = ds["validation"]
test_ds  = ds["test"]

tmp_dataset = FlickrDataset(
    train_ds,
    vocab_path="/content/vocab-HF-8K.pkl"
)

VOCAB_SIZE = len(tmp_dataset.word2int)


# 3. Initialize model (M1)
model = ResNetLSTMCaptionModel(
    vocab_size=VOCAB_SIZE,
    embed_dim=256,
    hidden_dim=512
)


# 4. Train + validate + test
history = fit_model_hf(
    model=model,
    train_ds=train_ds,
    val_ds=val_ds,
    test_ds=test_ds,
    vocab_path="/content/vocab-HF-8K.pkl",
    output_model_path="/content/m1_resnet_lstm.pth",
    batch_size=32,
    num_epochs=5,
    lr=1e-4,
    device=torch.device("cuda")
)


Epoch 1/5 | Train: 5.6467 | Val: 4.3166 | Test: 4.3120
Epoch 2/5 | Train: 4.2533 | Val: 4.0056 | Test: 4.0059
Epoch 3/5 | Train: 3.9857 | Val: 3.8201 | Test: 3.8189
Epoch 4/5 | Train: 3.8127 | Val: 3.6944 | Test: 3.6897
Epoch 5/5 | Train: 3.6855 | Val: 3.5985 | Test: 3.5891
Model saved to: /content/m1_resnet_lstm.pth
